In [2]:
import pandas as pd
import os
#cerrega todo os arquivos
dfsNew = []
dfsOld = []

for arq in os.listdir("./data/CHN/"):
    if arq.find("New") > 0:
        dfsNew.append(pd.read_csv("./data/CHN/"+arq, sep="\t"))
    elif arq.find("Old") > 0:
        dfsOld.append(pd.read_csv("./data/CHN/"+arq, sep="\t"))


In [14]:
from nltk import ngrams
from sacremoses import MosesTokenizer

def tokenize(text, lang, nGram=3):
    entok = MosesTokenizer(lang=lang)
    text = entok.tokenize(text, escape=False)
    grams = []
    for i in range(1, nGram):
        i_grams = [
            " ".join(gram)
            for gram in ngrams(text, i)
        ]
        grams.extend(i_grams)
        
    return grams

def getNgramOverlap(hypothesys, references, nGram, lang):

  overlaps = []
  for h, r in zip([hypothesys], [references]):
    if (h == "") or (r == ""):
      overlaps.append(1.0)
      continue
    a = tokenize(h, lang, nGram)
    b = tokenize(r, lang, nGram)

    if len(a) >= len(b) and len(a) > 0:
      overlaps.append(len(set(a) & set(b))/len(a))
    elif len(b) >= len(a) and len(b) > 0:
      overlaps.append(len(set(a) & set(b))/len(b))
    elif len(a) == 0 or len(b) == 0:
      overlaps.append(0)

  return overlaps[0]

import numpy as np

def getStats(dfA, dfB, lang):
    df = dfA.merge(dfB, on=["livro", "capitulo", "versiculo"])

    df["sourceLen"] = df["texto_x"].apply(lambda x: getSizeSentece(x))
    df["targetLen"] = df["texto_y"].apply(lambda x: getSizeSentece(x))
    
    df["overlap"] = df.apply(lambda x: getNgramOverlap(x["texto_x"], x["texto_y"], 3, lang), axis=1)

    return df
#Matriz do novo testamento

def getSizeSentece(text):
    try:
        return len(text.split(" "))
    except:
        return 0
    


In [15]:
print("Novo testamento: ")
newConcat = []
for dfA in dfsNew:
    dfsNew = dfsNew[1:]
    for dfB in dfsNew:
        newConcat.append(getStats(dfA, dfB, "zh"))

Novo testamento: 


In [16]:
dfConcat = pd.concat(newConcat, ignore_index=False)
print("tamanho sem cortes: ", dfConcat.shape)
#remove versiculos deseconhecidos
dfConcat = dfConcat[dfConcat["versiculo"] <= 180]
#remove textos muito difenretes
dfConcat = dfConcat[dfConcat["overlap"] > 0.02]
#filtra sentencas curtas
dfConcat = dfConcat[dfConcat["sourceLen"] > 5]
dfConcat = dfConcat[dfConcat["targetLen"] > 5]

print("tamanho com cortes: ", dfConcat.shape)

tamanho sem cortes:  (221744, 10)


TypeError: '<=' not supported between instances of 'str' and 'int'

In [ ]:
dfConcat.to_csv("./data/CHN/NovoTestamentoCompleto-CHN.tsv", sep="\t", index=False)

In [ ]:
print("Velho testamento: ")
oldConcat = []
for dfA in dfsOld:
    dfsOld = dfsOld[1:]
    for dfB in dfsOld:
        oldConcat.append(getStats(dfA, dfB, "zh"))

Velho testamento: 


In [ ]:
dfConcat = pd.concat(oldConcat, ignore_index=False)
print("tamanho sem cortes: ", dfConcat.shape)
#remove versiculos deseconhecidos
dfConcat = dfConcat[dfConcat["versiculo"] <= 180]
#remove textos muito difenretes
dfConcat = dfConcat[dfConcat["overlap"] > 0.02]
#filtra sentencas curtas
dfConcat = dfConcat[dfConcat["sourceLen"] > 5]
dfConcat = dfConcat[dfConcat["targetLen"] > 5]

print("tamanho com cortes: ", dfConcat.shape)

tamanho sem cortes:  (10246620, 10)
tamanho com cortes:  (8841448, 10)


In [ ]:
dfConcat.to_csv("./data/CHN/VelhoTestamentoCompleto-CHN.tsv", sep="\t", index=False)